In [8]:
from pyspark.sql import SparkSession

In [9]:
spark = SparkSession.builder.appName('lr_example').getOrCreate()

your 131072x1 screen size is bogus. expect trouble
23/09/07 13:21:35 WARN Utils: Your hostname, Javier-Garcia resolves to a loopback address: 127.0.1.1; using 192.168.145.121 instead (on interface eth0)
23/09/07 13:21:35 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/09/07 13:21:38 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [10]:
from pyspark.ml.regression import LinearRegression

In [11]:
data = spark.read.csv('Ecommerce_Customers.csv', header=True, inferSchema=True)

In [12]:
data.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [13]:
data.show()

+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
|               Email|             Address|          Avatar|Avg Session Length|       Time on App|   Time on Website|Length of Membership|Yearly Amount Spent|
+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
|mstephenson@ferna...|835 Frank TunnelW...|          Violet| 34.49726772511229| 12.65565114916675| 39.57766801952616|  4.0826206329529615|  587.9510539684005|
|   hduke@hotmail.com|4547 Archer Commo...|       DarkGreen| 31.92627202636016|11.109460728682564|37.268958868297744|    2.66403418213262|  392.2049334443264|
|    pallen@yahoo.com|24645 Valerie Uni...|          Bisque|33.000914755642675|11.330278057777512|37.110597442120856|   4.104543202376424| 487.54750486747207|
|riverarebecca@gma...|1414 David Throug...|   

In [14]:
from pyspark.ml.linalg import Vector
from pyspark.ml.feature import VectorAssembler

In [15]:
data.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [16]:
assembler = VectorAssembler(inputCols=['Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership'], outputCol='features')

In [17]:
output = assembler.transform(data)

In [18]:
output.head(1)

[Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005, features=DenseVector([34.4973, 12.6557, 39.5777, 4.0826]))]

In [19]:
final_data = output.select('features', 'Yearly Amount Spent')

In [20]:
final_data.show()

+--------------------+-------------------+
|            features|Yearly Amount Spent|
+--------------------+-------------------+
|[34.4972677251122...|  587.9510539684005|
|[31.9262720263601...|  392.2049334443264|
|[33.0009147556426...| 487.54750486747207|
|[34.3055566297555...|  581.8523440352177|
|[33.3306725236463...|  599.4060920457634|
|[33.8710378793419...|   637.102447915074|
|[32.0215955013870...|  521.5721747578274|
|[32.7391429383803...|  549.9041461052942|
|[33.9877728956856...|  570.2004089636196|
|[31.9365486184489...|  427.1993848953282|
|[33.9925727749537...|  492.6060127179966|
|[33.8793608248049...|  522.3374046069357|
|[29.5324289670579...|  408.6403510726275|
|[33.1903340437226...|  573.4158673313865|
|[32.3879758531538...|  470.4527333009554|
|[30.7377203726281...|  461.7807421962299|
|[32.1253868972878...| 457.84769594494855|
|[32.3388993230671...| 407.70454754954415|
|[32.1878120459321...|  452.3156754800354|
|[32.6178560628234...|   605.061038804892|
+----------

In [21]:
train_data, test_data = final_data.randomSplit([0.7, 0.3])

In [22]:
train_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                337|
|   mean|  499.6568341618466|
| stddev|  79.37361067319561|
|    min| 256.67058229005585|
|    max|  744.2218671047146|
+-------+-------------------+



In [23]:
lr = LinearRegression(labelCol='Yearly Amount Spent')

In [24]:
lr_model = lr.fit(train_data)

23/09/07 13:21:45 WARN Instrumentation: [4b1eff7d] regParam is zero, which might cause numerical instability and overfitting.
23/09/07 13:21:46 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
23/09/07 13:21:46 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


In [25]:
test_results = lr_model.evaluate(test_data)

In [26]:
test_results.residuals.show()

+--------------------+
|           residuals|
+--------------------+
|    9.49223813144215|
|-0.34626145057467284|
|  -4.079996875570771|
| -3.2412474205548847|
|  -6.536089314809146|
|  10.955657864335421|
| -4.2804335108322675|
|  22.344277590289835|
|   2.690623612326135|
|  0.3728083465158534|
|  -5.764601285294248|
| -3.9550130475772107|
|   7.512111752205385|
| -26.054373675618308|
| -3.9014789538852597|
|  -4.634869970110344|
|  -16.39011156701571|
|   7.097825283893883|
|    8.37135384627885|
| -0.5408891733895871|
+--------------------+
only showing top 20 rows



In [27]:
test_results.rootMeanSquaredError

10.54648618344205

In [28]:
test_results.r2

0.9822626200944987

In [29]:
final_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                500|
|   mean|  499.3140382585909|
| stddev|   79.3147815497068|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



In [30]:
unlabeled_data = test_data.select('features')

In [31]:
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|[29.5324289670579...|
|[30.5743636841713...|
|[30.8364326747734...|
|[30.8794843441274...|
|[31.1280900496166...|
|[31.1695067987115...|
|[31.2681042107507...|
|[31.2834474760581...|
|[31.3091926408918...|
|[31.3895854806643...|
|[31.5257524169682...|
|[31.6253601348306...|
|[31.6548096756927...|
|[31.6739155032749...|
|[31.7656188210424...|
|[31.8124825597242...|
|[31.8164283341993...|
|[31.8209982016720...|
|[31.8512531286083...|
|[31.8627411090001...|
+--------------------+
only showing top 20 rows



In [32]:
predictions = lr_model.transform(unlabeled_data)

In [33]:
predictions.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[29.5324289670579...|399.14811294118533|
|[30.5743636841713...| 442.4106752086403|
|[30.8364326747734...| 471.5818973025604|
|[30.8794843441274...|493.44784740540956|
|[31.1280900496166...| 563.7887760618638|
|[31.1695067987115...| 416.4008729379574|
|[31.2681042107507...| 427.7509666846562|
|[31.2834474760581...| 569.4368118353776|
|[31.3091926408918...| 430.0300942276075|
|[31.3895854806643...|409.69680271346704|
|[31.5257524169682...|449.73022809517613|
|[31.6253601348306...| 380.2919138045014|
|[31.6548096756927...| 467.7513119753431|
|[31.6739155032749...| 501.7794415854995|
|[31.7656188210424...| 500.4555605894924|
|[31.8124825597242...|397.44521495390757|
|[31.8164283341993...| 517.5126030706721|
|[31.8209982016720...|417.57745572931947|
|[31.8512531286083...|464.62089282051954|
|[31.8627411090001...| 556.8390303474363|
+--------------------+------------